In [1]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np
import PIL
from random import shuffle
from skimage.transform import resize
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [2]:
import pickle

In [3]:
with open('dataconv.pt', 'rb') as f:
    data = pickle.load(f)
    
with open('labels.pt', 'rb') as f:
    labels = pickle.load(f)

In [4]:
class VGG16(nn.Module):
    def __init__(self,dropout=0.5):
        super(VGG16, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3)
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3)
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3)
        self.conv7 = nn.Conv2d(256, 512, kernel_size=3)
        self.conv8 = nn.Conv2d(512, 512, kernel_size=3)
        self.conv9 = nn.Conv2d(512, 512, kernel_size=3,padding=1)
        self.mp = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(3072,4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 4)        
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        in_size = x.size(0)
        x = self.mp(x)
        x = self.dropout(torch.relu(self.conv1(x)))
        x = self.dropout(torch.relu(self.conv2(x)))
        x = self.mp(x)
        x = self.dropout(torch.relu(self.conv3(x)))
        x = self.dropout(torch.relu(self.conv4(x)))
        x = self.mp(x)
        x = self.dropout(torch.relu(self.conv5(x)))
        x = self.dropout(torch.relu(self.conv6(x)))
        x = self.dropout(torch.relu(self.conv6(x)))
        x = self.mp(x)
        x = self.dropout(torch.relu(self.conv7(x)))
        x = self.dropout(torch.relu(self.conv8(x)))
        x = self.dropout(torch.relu(self.conv8(x)))
        x = self.mp(x)
        x = self.dropout(torch.relu(self.conv8(x)))
        x = self.dropout(torch.relu(self.conv8(x)))
        x = self.dropout(torch.relu(self.conv9(x)))
        x = self.mp(x)
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.dropout(torch.relu(self.fc1(x)))
        x = self.dropout(torch.relu(self.fc2(x)))
        x = self.fc3(x)
        return torch.log_softmax(x,1)


In [ ]:
class AutoUXDataset(Dataset):
    def __init__(self, data, labels,isTrain=True):
        self.data = data
        self.labels = labels
        self.isTrain = isTrain
        self.setTest(0)

    def __len__(self):
        if self.isTrain:
            return len(self.trainX)
        else:
            return len(self.testX)

    def setTest(self,testI):
        d = []
        l = []
        for i in range(5):
            if i != testI:
                d = d + data[i];
                l = l + labels[i]
            else:
                self.testX = torch.from_numpy(np.asarray(data[i],dtype=np.uint8)).type('torch.FloatTensor')
                self.testY = torch.from_numpy(np.asarray(labels[i],dtype=np.uint8)).type('torch.LongTensor')
                
        self.trainX = torch.from_numpy(np.asarray(d,dtype=np.uint8)).type('torch.FloatTensor')
        self.trainY = torch.from_numpy(np.asarray(l,dtype=np.uint8)).type('torch.LongTensor')
    
    def __getitem__(self, idx):
        if self.isTrain:
            return {"img" : self.trainX[idx], "label": self.trainY[idx]}
        else:
            return {"img" : self.testX[idx], "label": self.testY[idx]}

In [ ]:
def train_batch(model,crit,opt,x, y):
    opt.zero_grad()
    y_pred = model(x)
    loss = crit(y_pred, y)
    loss.backward()
    opt.step() 
    return loss.item(),torch.sum(torch.max(y_pred, 1)[1] == y).item() / len(y)

def train(model,crit,opt,dataload,disable=False):
    # Training loop
    dloss, dacc = 0,0
    for s in tqdm(dataload,disable=disable):
        x, y = s.values()
        x = x.cuda()
        y = y.cuda()
        lo, acc = train_batch(model,crit,opt,x,y)
        dloss += lo
        dacc += acc
    dloss /= len(dataload)
    dacc /= len(dataload)
    return dloss,dacc

def test_batch(model,crit,x, y):
    y_pred = model(x)
    loss = crit(y_pred, y)
    return loss.item(),torch.sum(torch.max(y_pred, 1)[1] == y).item() / len(y)

def test(model,crit,dataload,disable=False):
    # Training loop
    dloss, dacc = 0,0
    with torch.no_grad():
        for s in tqdm(dataload,disable=disable):
            x, y = s.values()
            x = x.cuda()
            y = y.cuda()
            lo, acc = test_batch(model,crit,x,y)
            dloss += lo
            dacc += acc
    dloss /= len(dataload)
    dacc /= len(dataload)
    return dloss,dacc

In [ ]:
avedloss, avedacc, avevloss, avevacc = 0,0,0,0
my_data = AutoUXDataset(data,labels,True)
my_test = AutoUXDataset(data,labels,False)

batch_size = 32
disable = False
epochs = 50
dropout = 0.3
weight_decay = 0.075

for i in range(5):
    model = VGG16(dropout=dropout).cuda()
    optimizer = optim.Adam(model.parameters(), lr=3e-4,weight_decay=weight_decay)
    #amp_handle.wrap_optimizer(optimizer)
    criterion = nn.CrossEntropyLoss()
    #amp_handle = amp.init()
    dlosscurve = []
    dacccurve = []
    vlosscurve = []
    vacccurve = []
    print("Fold %d" % (i + 1))
    my_data.setTest(i)
    my_test.setTest(i)
    
    my_loader = DataLoader(my_data, batch_size=batch_size,
                            shuffle=True)
    my_test_loader = DataLoader(my_test, batch_size=batch_size,
                            shuffle=True)
    for j in range(epochs):
        dloss, dacc = train(model,criterion,optimizer,my_loader,disable)
        vloss, vacc = test(model,criterion,my_test_loader,disable)
        dlosscurve.append(dloss)
        dacccurve.append(dacc)
        vlosscurve.append(vloss)
        vacccurve.append(vacc)
        if j % 1 == 0:
            print("Epoch Train Loss: {:.6f}  Epoch Train Accuracy: {:.6f}  Epoch Test Loss: {:.6f}  Epoch Test Accuracy: {:.6f}".format(dloss,dacc * 100,vloss,vacc * 100))
    df = pd.DataFrame(data={"train_loss": dlosscurve, "val_loss": vlosscurve})
    df.plot.line()
    avedloss += dloss
    avedacc += dacc
    avevloss += vloss
    avevacc += vacc
avedloss /= 5
avedacc /= 5
avevloss /= 5
avevacc /= 5
print("Average Train Loss: {:.6f}\nAverage Train Accuracy: {:.6f}\nAverage Test Loss: {:.6f}\nAverage Test Accuracy: {:.6f}\n".format(avedloss,avedacc,avevloss,avevacc))

Fold 1


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  3.94it/s]


Epoch Train Loss: 1.151829  Epoch Train Accuracy: 50.287829  Epoch Test Loss: 0.843656  Epoch Test Accuracy: 48.505435


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  5.70it/s]


Epoch Train Loss: 0.746913  Epoch Train Accuracy: 51.891447  Epoch Test Loss: 0.701716  Epoch Test Accuracy: 46.512681


...

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  5.75it/s]


Epoch Train Loss: 0.703906  Epoch Train Accuracy: 51.158169  Epoch Test Loss: 0.703893  Epoch Test Accuracy: 48.278986


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  5.48it/s]


Epoch Train Loss: 0.701281  Epoch Train Accuracy: 49.067982  Epoch Test Loss: 0.701439  Epoch Test Accuracy: 48.075181


 46%|█████████████████████████████████████▌                                            | 11/24 [00:05<00:06,  2.08it/s]

In [ ]:
class AutoUXFullDataset(Dataset):
    def __init__(self, data, labels):
        super(AutoUXFullDataset,self).__init__()
        self.data = data
        self.labels = labels
        d = []
        l = []
        for i in range(5):
            d = d + data[i];
            l = l + labels[i]
        self.trainX = torch.from_numpy(np.asarray(d,dtype=np.uint8)).type('torch.FloatTensor')
        self.trainY = torch.from_numpy(np.asarray(l,dtype=np.uint8)).type('torch.LongTensor')

    def __len__(self):
        return len(self.trainX)
    
    def __getitem__(self, idx):
        return {"img" : self.trainX[idx], "label": self.trainY[idx]}

In [ ]:
avedloss, avedacc = 0,0
my_data = AutoUXFullDataset(data,labels)

batch_size = 32
disable = False
epochs = 50
every_other = 1

model = VGG16(0.5).cuda()
optimizer = optim.Adam(model.parameters(), lr=3e-4,weight_decay=0.1)
#amp_handle.wrap_optimizer(optimizer)
criterion = nn.CrossEntropyLoss()
#amp_handle = amp.init()
dlosscurve = []
dacccurve = []

my_loader = DataLoader(my_data, batch_size=batch_size,
                        shuffle=True)
for j in range(epochs):
    dloss, dacc = train(model,criterion,optimizer,my_loader,disable)
    dlosscurve.append(dloss)
    dacccurve.append(dacc)
    if j % every_other == 0:
        print("Epoch Train Loss: {:.6f}  Epoch Train Accuracy: {:.6f}".format(dloss,dacc * 100))
df = pd.DataFrame(data={"train_loss": dlosscurve})
df.plot.line()
avedloss += dloss
avedacc += dacc
print("Average Train Loss: {:.6f}\nAverage Train Accuracy: {:.6f}".format(avedloss,avedacc))

In [ ]:
my_loader = DataLoader(my_data, batch_size=32,
                        shuffle=False)
answers = np.array([])
with torch.no_grad():
    for s in tqdm(my_loader,disable=False):
        x, y = s.values()
        x = x.cuda()
        y = y.cuda()
        y_pred = model(x).cpu()
        answers = np.append(answers,y_pred[:,1])
answers = np.exp(answers)
print(answers)

In [ ]:
with open('VGG16outprobs.pt', 'wb') as f:
    pickle.dump(answers, f)

In [ ]:
from sklearn.metrics import *
import itertools
import pickle
with open('VGG16out.pt', 'rb') as f:
    y_pred = pickle.load(f)
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()


y_test = np.array([])

for x in labels:
    y_test = np.append(y_test,x)

class_names = ["bad site","good site"]

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()
print("Accuracy: {:.4f}".format(np.mean(y_test==y_pred)))